In [1]:
!conda install -y -c conda-forge faiss-gpu
!apt-get -y update
!apt-get -y install libatlas-base-dev

done
Solving environment: done

## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - faiss-gpu


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.14.0               |   py37h89c1867_0        1010 KB  conda-forge
    toolz-0.12.1               |     pyhd8ed1ab_0          51 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         1.0 MB

The following NEW packages will be INSTALLED:

  toolz              conda-forge/noarch::toolz-0.12.1-pyhd8ed1ab_0

The following packages will be UPDATED:

  conda                               4.12.0-py37h89c1867_0 --> 4.14.0-py37h89c1867_0



conda-4.14.0         | 1010 KB   | ##################################### | 100% 
toolz-0.12.1         | 51 KB     | ##################################### | 100% 
Preparing transaction: done
Ver

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.random_projection import GaussianRandomProjection

from tqdm import tqdm

import faiss

In [3]:
df = pd.read_csv("story_dataset.csv")
df

,prompt_id,prompt,story,hidden_state_file,len_generated_story,len_new_story
0,1,Once upon a time there was a dragon,Once upon a time there was a dragon named Blaz...,./hidden_states/prompt_1.npz,270,271
1,1,Once upon a time there was a dragon,Once upon a time there was a dragon named Spar...,./hidden_states/prompt_1.npz,349,350
2,1,Once upon a time there was a dragon,Once upon a time there was a dragon named Scor...,./hidden_states/prompt_1.npz,278,278
3,1,Once upon a time there was a dragon,Once upon a time there was a dragon. The drago...,./hidden_states/prompt_1.npz,117,118
4,1,Once upon a time there was a dragon,Once upon a time there was a dragon. The drago...,./hidden_states/prompt_1.npz,129,130
...,...,...,...,...,...,...
9995,10,Once upon a time there was a poor boy,Once upon a time there was a poor boy named Ti...,./hidden_states/prompt_10.npz,289,290
9996,10,Once upon a time there was a poor boy,Once upon a time there was a poor boy named Ti...,./hidden_states/prompt_10.npz,119,119
9997,10,Once upon a time there was a poor boy,Once upon a time there was a poor boy named Ti...,./hidden_states/prompt_10.npz,127,128
9998,10,Once upon a time there was a poor boy,Once upon a time there was a poor boy named Ti...,./hidden_states/prompt_10.npz,441,441


In [4]:
max_story_len = max(df["len_generated_story"])
max_story_len

522

In [5]:
hidden_states_by_layer = {}
NUM_PROMPTS = 2

# for prompt_id in range(1, 11):
for prompt_id in range(1, NUM_PROMPTS + 1):
    with np.load(f'./hidden_states/prompt_{prompt_id}.npz') as loaded_data:
        for i in tqdm(range(1000)):
            curr_hidden_states = loaded_data[f"arr_{i}"][0]
#             print(curr_hidden_states.shape)
            for layer in range(1):
                padded_arr = np.zeros((max_story_len, 512))
                padded_arr_len = len(curr_hidden_states[layer][0])
                
                padded_arr[:padded_arr_len] = curr_hidden_states[layer][0]
                
                padded_arr = padded_arr.flatten().astype('float32') #FAISS expects data in type float32 instead of float64 - saves memory too!
#                 print(padded_arr.shape)
                
                if(f"layer_{layer}" in hidden_states_by_layer):
                    hidden_states_by_layer[f"layer_{layer}"].append(padded_arr)
                else:
                    hidden_states_by_layer[f"layer_{layer}"] = [padded_arr]

100%|██████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:24<00:00, 40.16it/s]


In [6]:
np.array(hidden_states_by_layer["layer_0"]).shape

(2000, 267264)

In [7]:
# Layer 0 clustering

random_projector = GaussianRandomProjection(random_state = 42)
dim_reduced_vecs = random_projector.fit_transform(np.array(hidden_states_by_layer["layer_0"])).astype('float32')

In [8]:
dim_reduced_vecs.shape

(2000, 6515)

In [9]:
# K-means Clustering

ncentroids = NUM_PROMPTS
niter = 50
verbose = True
dim = dim_reduced_vecs.shape[1]
kmeans = faiss.Kmeans(dim, ncentroids, niter=niter, verbose=verbose, gpu=True, nredo = 20)
kmeans.train(dim_reduced_vecs)

Sampling a subset of 512 / 2000 for training
Clustering 512 points in 6515D to 2 clusters, redo 20 times, 50 iterations
  Preprocessing in 0.01 s
Outer iteration 0 / 20
  Iteration 49 (0.08 s, search 0.04 s): objective=167733 imbalance=1.886 nsplit=0       
Objective improved: keep new clusters
Outer iteration 1 / 20
  Iteration 49 (0.15 s, search 0.08 s): objective=167812 imbalance=1.992 nsplit=0       
Outer iteration 2 / 20


  Iteration 49 (0.22 s, search 0.11 s): objective=167751 imbalance=1.992 nsplit=0       
Outer iteration 3 / 20
  Iteration 49 (0.29 s, search 0.15 s): objective=167749 imbalance=1.992 nsplit=0       
Outer iteration 4 / 20
  Iteration 49 (0.37 s, search 0.19 s): objective=167699 imbalance=1.984 nsplit=0       
Objective improved: keep new clusters
Outer iteration 5 / 20


  Iteration 49 (0.44 s, search 0.23 s): objective=167700 imbalance=1.992 nsplit=0       
Outer iteration 6 / 20
  Iteration 49 (0.51 s, search 0.26 s): objective=167722 imbalance=1.992 nsplit=0       
Outer iteration 7 / 20
  Iteration 49 (0.58 s, search 0.30 s): objective=167630 imbalance=1.992 nsplit=0       
Objective improved: keep new clusters
Outer iteration 8 / 20


  Iteration 49 (0.66 s, search 0.34 s): objective=167760 imbalance=1.992 nsplit=0       
Outer iteration 9 / 20
  Iteration 49 (0.73 s, search 0.37 s): objective=167753 imbalance=1.992 nsplit=0       
Outer iteration 10 / 20
  Iteration 49 (0.80 s, search 0.41 s): objective=167706 imbalance=1.992 nsplit=0       
Outer iteration 11 / 20


  Iteration 49 (0.87 s, search 0.45 s): objective=168158 imbalance=1.992 nsplit=0       
Outer iteration 12 / 20
  Iteration 49 (0.94 s, search 0.49 s): objective=167964 imbalance=1.992 nsplit=0       
Outer iteration 13 / 20
  Iteration 49 (1.01 s, search 0.53 s): objective=167744 imbalance=1.908 nsplit=0       
Outer iteration 14 / 20


  Iteration 49 (1.09 s, search 0.56 s): objective=167638 imbalance=1.992 nsplit=0       
Outer iteration 15 / 20
  Iteration 49 (1.16 s, search 0.60 s): objective=167730 imbalance=1.992 nsplit=0       
Outer iteration 16 / 20


  Iteration 49 (1.23 s, search 0.64 s): objective=167828 imbalance=1.992 nsplit=0       
Outer iteration 17 / 20
  Iteration 49 (1.30 s, search 0.67 s): objective=167658 imbalance=1.992 nsplit=0       
Outer iteration 18 / 20
  Iteration 49 (1.37 s, search 0.71 s): objective=167674 imbalance=1.992 nsplit=0       
Outer iteration 19 / 20


167629.703125

In [10]:
kmeans.centroids #cluster centers

array([[ 0.03878354,  0.05904993,  0.0282885 , ..., -0.03968159,
        -0.02476441, -0.05587641],
       [ 0.01763256, -0.21713637,  0.29737765, ...,  0.00347101,
         0.01532576,  0.01871864]], dtype=float32)

In [11]:
kmeans.obj #inertia at each iteration

array([232817.984375, 167796.890625, 167773.9375  , 167743.390625,
       167732.9375  , 167732.9375  , 167732.9375  , 167732.9375  ,
       167732.9375  , 167732.9375  , 167732.9375  , 167732.9375  ,
       167732.9375  , 167732.9375  , 167732.9375  , 167732.9375  ,
       167732.9375  , 167732.9375  , 167732.9375  , 167732.9375  ,
       167732.9375  , 167732.9375  , 167732.9375  , 167732.9375  ,
       167732.9375  , 167732.9375  , 167732.9375  , 167732.9375  ,
       167732.9375  , 167732.9375  , 167732.9375  , 167732.9375  ,
       167732.9375  , 167732.9375  , 167732.9375  , 167732.9375  ,
       167732.9375  , 167732.9375  , 167732.9375  , 167732.9375  ,
       167732.9375  , 167732.9375  , 167732.9375  , 167732.9375  ,
       167732.9375  , 167732.9375  , 167732.9375  , 167732.9375  ,
       167732.9375  , 167732.9375  , 233874.84375 , 167812.5     ,
       167812.5     , 167812.5     , 167812.5     , 167812.5     ,
       167812.5     , 167812.5     , 167812.5     , 167812.5  

In [12]:
pd.Series(kmeans.index.search(dim_reduced_vecs.astype(np.float32), 1)[1].flatten()).value_counts()

0    1999
1       1
dtype: int64

In [13]:
classifications = []
for dim_red_vec in dim_reduced_vecs:
    dist_from_first_centroid = np.linalg.norm((dim_red_vec - kmeans.centroids[0]))
    dist_from_second_centroid = np.linalg.norm((dim_red_vec - kmeans.centroids[1]))
    classification = 0 if dist_from_first_centroid < dist_from_second_centroid else 1
    
    classifications.append(classification)
    
pd.Series(classifications).value_counts()

0    1999
1       1
dtype: int64